# Display 3D neuron with crystals

Load img

In [4]:
import nrrd
lab_n, _ = nrrd.read(r"D:\Zuohan\neuron\neuron.nrrd")
lab_c, _ = nrrd.read(r"D:\Zuohan\neuron\crystal.nrrd")

import yaml
from CrystalTracer3D.io import CrystalReader
with open('config.yml', 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    slice_range = cfg['data']['range']
reader = CrystalReader(cfg['data']['path'])
ch = reader.find_channel(cfg['data']['neuron'])
img_n = reader.read(ch)[slice_range[0]:slice_range[1]]
ch = reader.find_channel(cfg['data']['crystal'])
img_c = reader.read(ch)[slice_range[0]:slice_range[1]]


In [2]:
from skimage.segmentation import expand_labels
lab_n = expand_labels(lab_n, 1, spacing=reader.resolution)

# raw image only

In [5]:
import napari
import numpy as np
viewer = napari.Viewer(ndisplay=3)
layers = viewer.add_image(np.stack([img_n, img_c]), channel_axis=0, name=['neuron', 'crystal'], colormap=["green", "red"], blending=['translucent', "additive"], scale=reader.resolution)
layers[0].contrast_limits = [3587.828571428571, 14532.678571428572]
layers[1].contrast_limits = [6869.25, 38216.67857142857]
bbox = layers[0].bounding_box
bbox.visible = True
bbox.line_color = 'white'
bbox.line_thickness = 2
bbox.point_size = 0
bbox = layers[1].bounding_box
bbox.visible = True
bbox.line_color = 'white'
bbox.line_thickness = 2
bbox.point_size = 0
viewer.camera.perspective = 30
viewer.camera.zoom = 5
viewer.camera.angles = (-1.5, 20, 85)
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'μm'
viewer.scale_bar.length = 20
viewer.scale_bar.font_size *= 1.5

In [6]:
viewer.screenshot(path=r"D:\Zuohan\neuron\both_raw.png", canvas_only=True)
viewer.scale_bar.visible = False
viewer.screenshot(path=r"D:\Zuohan\neuron\both_raw_no_scale.png", canvas_only=True)
layers[0].visible = False
viewer.screenshot(path=r"D:\Zuohan\neuron\crystal_raw.png", canvas_only=True)
layers[1].visible = False
layers[0].visible = True
viewer.screenshot(path=r"D:\Zuohan\neuron\neuron_raw.png", canvas_only=True)
viewer.close()

# seg only

In [54]:
viewer = napari.Viewer(ndisplay=3)
layer = viewer.add_labels(lab_n, opacity=.8, scale=reader.resolution)
layer.iso_gradient_mode = 'smooth'
layer = viewer.add_labels(lab_c, opacity=.8, scale=reader.resolution)
layer.iso_gradient_mode = 'smooth'
layer = viewer.add_image(img_n, opacity=.7, blending='additive', scale=reader.resolution)
layer.contrast_limits = [3587.828571428571, 14532.678571428572]
bbox = layer.bounding_box
bbox.visible = True
bbox.line_color = 'white'
bbox.line_thickness = 2
bbox.point_size = 0
viewer.camera.zoom = 5
viewer.camera.perspective = 30
viewer.camera.angles = (-1.5, 20, 85)

In [55]:
viewer.layers[1].visible = False
viewer.screenshot(path=r"D:\Zuohan\neuron\neuron_seg.png", canvas_only=True)
viewer.layers[1].visible = True
viewer.layers[0].visible = False
viewer.screenshot(path=r"D:\Zuohan\neuron\crystal_seg.png", canvas_only=True)
viewer.close()

# Paired
Load pair info

In [52]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata
pairs = pd.read_csv(r"D:\Zuohan\neuron\pair.csv")
remove = [262, 3]
pairs = pairs[~pairs['neuron'].isin(remove)]
pairs['rank'] = rankdata(pairs['neuron'], method='min')
pairs = pairs.sort_values('rank')
pairs.to_csv(r"D:\Zuohan\neuron\pair_filtered.csv", index=False)

In [53]:
a1 = np.isin(lab_c, pairs['crystal'])
a2 = np.isin(lab_n, pairs['neuron'])

In [58]:
import napari
viewer = napari.Viewer(ndisplay=3)
layer = viewer.add_image(img_n, opacity=.7, blending='additive', scale=reader.resolution)
layer.contrast_limits = [3587.828571428571, 14532.678571428572]
viewer.add_labels(a1, name='crystal', opacity=.7, colormap={1: 'red', None: [0, 0, 0, 0]}, blending='additive', scale=reader.resolution)
viewer.add_labels(a2, name='neuron', opacity=.7, colormap={1: 'green', None: [0, 0, 0, 0]}, blending='additive', scale=reader.resolution)
viewer.layers[1].iso_gradient_mode = 'smooth'
viewer.layers[2].iso_gradient_mode = 'smooth'
bbox = layer.bounding_box
bbox.visible = True
bbox.line_color = 'white'
bbox.line_thickness = 2
bbox.point_size = 0
viewer.camera.perspective = 30
viewer.camera.zoom = 5
viewer.camera.angles = (-1.5, 20, 85)

adjust viz

In [59]:
viewer.screenshot(path=r"D:\Zuohan\neuron\pairs_all.png", canvas_only=True)
viewer.close()

# Number

In [60]:
neurons_df = pd.read_csv(r"D:\Zuohan\neuron\neuron_prop.csv", index_col='label')
n = np.unique(pairs['neuron'])
centers = neurons_df.loc[n, ['z', 'y', 'x']].to_numpy()

In [76]:
import napari
viewer = napari.Viewer(ndisplay=3)
layer = viewer.add_image(img_n, opacity=.7, blending='additive', scale=reader.resolution)
layer.contrast_limits = [3587.828571428571, 14532.678571428572]
viewer.add_labels(lab_n, opacity=.7, blending='additive', scale=reader.resolution, rendering='translucent')
viewer.layers[1].iso_gradient_mode = 'smooth'
# layer.bounding_box.visible = True
# layer.bounding_box.line_color = 'white'
# layer.bounding_box.line_thickness = 2
# layer.bounding_box.point_size = 0
viewer.add_points(
    neurons_df[['z', 'y', 'x']],
    features={'no': np.arange(len(neurons_df.index)) + 1},
    text={
        'string': '{no}',
        'size': 6,
        'scaling': True,
        'color': 'white',
        'translation': np.array([0, -20, 0]),
    },
    size=10,
    face_color='red', scale=reader.resolution,
)
viewer.camera.zoom = 5
viewer.camera.angles = (0, 0, 90)
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'μm'
viewer.scale_bar.length = 20
viewer.scale_bar.font_size *= 1.5

In [77]:
viewer.screenshot(path=r"D:\Zuohan\neuron\numbers.png", canvas_only=True)
viewer.scale_bar.visible = False
viewer.screenshot(path=r"D:\Zuohan\neuron\numbers_no_scale.png", canvas_only=True)
viewer.close()